In [52]:
from google.colab import drive
drive.mount('/content/drive')

# Set dataset path in Google Drive
DATASET_PATH = "/content/drive/MyDrive/DataSet"
ANNOTATIONS_PATH = f"{DATASET_PATH}/annotations"
IMAGES_PATH = f"{DATASET_PATH}/images"
YOLO_LABELS_PATH = f"{DATASET_PATH}/labels"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
!pip install ultralytics
!pip install xmltodict tqdm opencv-python

import os
import xmltodict
import glob
import shutil
import cv2
from tqdm import tqdm

# Create YOLO labels directory
os.makedirs(YOLO_LABELS_PATH, exist_ok=True)

In [46]:
def convert_voc_to_yolo(xml_file, img_width, img_height):
    with open(xml_file, 'r') as file:
        doc = xmltodict.parse(file.read())

    objects = doc['annotation'].get('object', [])
    if not isinstance(objects, list):
        objects = [objects]  # Convert single object to list

    yolo_lines = []
    for obj in objects:
        class_name = obj['name']
        bbox = obj['bndbox']

        # Extract coordinates
        xmin, ymin, xmax, ymax = map(int, [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']])

        # Convert to YOLO format (x_center, y_center, width, height)
        x_center = (xmin + xmax) / (2 * img_width)
        y_center = (ymin + ymax) / (2 * img_height)
        bbox_width = (xmax - xmin) / img_width
        bbox_height = (ymax - ymin) / img_height

        # Assuming class 0 for potholes
        yolo_lines.append(f"0 {x_center} {y_center} {bbox_width} {bbox_height}")

    return yolo_lines

# Convert all XML annotations to YOLO format
for xml_file in tqdm(glob.glob(f"{ANNOTATIONS_PATH}/*.xml")):
    image_name = os.path.splitext(os.path.basename(xml_file))[0] + ".png"
    image_path = os.path.join(IMAGES_PATH, image_name)

    if not os.path.exists(image_path):
        continue

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    yolo_labels = convert_voc_to_yolo(xml_file, img_width, img_height)

    label_file = os.path.join(YOLO_LABELS_PATH, os.path.splitext(os.path.basename(xml_file))[0] + ".txt")
    with open(label_file, "w") as f:
        f.write("\n".join(yolo_labels))

print("✅ Annotations converted successfully!")

100%|██████████| 666/666 [00:29<00:00, 22.32it/s]

✅ Annotations converted successfully!


In [47]:
yaml_content = f"""path: {DATASET_PATH}
train: images
val: images
nc: 1
names: ['pothole']
"""

yaml_path = os.path.join(DATASET_PATH, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ dataset.yaml created successfully!")

✅ dataset.yaml created successfully!


In [48]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Using YOLOv8 nano model for efficiency
results = model.train(data=yaml_path, epochs=50, imgsz=640, batch=16)

print("✅ Training complete!")

100%|██████████| 6.25M/6.25M [00:00<00:00, 56.3MB/s]


Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/DataSet/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.5MB/s]


AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/DataSet/labels... 665 images, 30 backgrounds, 0 corrupt: 100%|██████████| 695/695 [00:10<00:00, 69.29it/s] 


...

      50/50      2.19G       0.89     0.6521      1.022         21        640: 100%|██████████| 44/44 [00:22<00:00,  1.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:15<00:00,  1.45it/s]

                   all        695       1740      0.905      0.898      0.945      0.739



50 epochs completed in 0.558 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:16<00:00,  1.36it/s]


                   all        695       1740      0.906      0.898      0.945      0.739
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train
✅ Training complete!


In [54]:
import shutil
shutil.copy("runs/detect/train/weights/best.pt", "/content/drive/MyDrive/yolo_pothole_model.pt")

'/content/drive/MyDrive/yolo_pothole_model.pt'